In [1]:
import logging
import argparse
import sys
import asyncio
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import os

import syft as sy
from syft.workers import websocket_client
from syft.workers.websocket_client import WebsocketClientWorker
from syft.frameworks.torch.fl import utils
from my_utils import MyWebsocketClientWorker, model_to_device, model_federated
import my_utils

LOG_INTERVAL = 25
logger = logging.getLogger("run_websocket_client")
# loss = nn.CrossEntropyLoss()

In [2]:
import run_websocket_client

In [3]:
hook = sy.TorchHook(torch)

In [4]:
raspi = {"host": "192.168.3.4", "hook": hook}
jetson_nano = {"host": "192.168.3.5", "hook": hook}

In [5]:
worker_a = MyWebsocketClientWorker(id='A', port=9292, **jetson_nano)

In [6]:
worker_b = MyWebsocketClientWorker(id='B', port=9292, **raspi)

In [7]:
worker_instances = [worker_a, worker_b]
client_devices = ['cuda', 'cpu']

In [8]:
for worker in worker_instances:
    worker.clear_objects_remote()

In [9]:
model_1 = run_websocket_client.ConvNet1D(input_size=400, num_classes=7)
traced_model_1 = torch.jit.trace(model_1, torch.zeros([1, 400, 3], dtype=torch.float))

In [10]:
model_2 = run_websocket_client.ConvNet1D(input_size=400, num_classes=7)
traced_model_2 = torch.jit.trace(model_2, torch.zeros([1, 400, 3], dtype=torch.float))
model_3 = run_websocket_client.ConvNet1D(input_size=400, num_classes=7)
traced_model_3 = torch.jit.trace(model_3, torch.zeros([1, 400, 3], dtype=torch.float))

In [11]:
traced_model_1.conv1.weight.data[0,0,:]

tensor([ 0.0359, -0.1432, -0.0946])

In [12]:
traced_model_2.conv1.weight.data[0,0,:]

tensor([-0.0968,  0.1780, -0.3004])

In [13]:
traced_model_1.conv1.weight.data[0,0,:] + traced_model_2.conv1.weight.data[0,0,:]

tensor([-0.0609,  0.0348, -0.3950])

In [14]:
traced_model_1.send(worker_a)
traced_model_2.send(worker_a)

ConvNet1D(
  original_name=ConvNet1D
  (conv1): Conv1d(original_name=Conv1d)
  (pool): MaxPool1d(original_name=MaxPool1d)
  (fc1): Linear(original_name=Linear)
  (fc2): Linear(original_name=Linear)
)

In [15]:
traced_model_3.conv1.weight.data

tensor([[[ 2.0389e-02,  2.3583e-02,  6.6996e-02],
         [-1.6800e-01,  2.2987e-01, -2.3620e-01],
         [-2.7821e-01,  3.2265e-01, -3.7342e-02]],

        [[-1.1650e-01,  1.3057e-01, -2.6335e-01],
         [ 3.3119e-01,  4.4344e-02,  2.5557e-01],
         [-4.1669e-02,  1.4127e-01,  1.1641e-01]],

        [[ 3.1050e-01, -3.0225e-01,  3.1629e-01],
         [ 1.5597e-01, -1.5927e-01,  1.2084e-01],
         [-3.0632e-01, -3.9383e-02,  2.0956e-01]],

        [[ 2.0772e-01,  2.0705e-02, -2.4642e-01],
         [ 2.7372e-01, -3.2175e-01,  1.0681e-01],
         [ 2.4934e-01,  3.1727e-01,  4.3778e-02]],

        [[-1.8393e-01, -6.2944e-02, -2.1750e-01],
         [ 6.8667e-02,  1.1321e-01,  2.7901e-01],
         [ 1.9587e-02,  1.2802e-03, -1.5364e-01]],

        [[-2.7912e-01,  1.0528e-01, -2.0015e-01],
         [-2.8628e-01, -8.1481e-02,  6.0222e-02],
         [-3.2275e-01, -2.2598e-01, -2.8433e-01]],

        [[-2.5045e-01,  3.2074e-01, -1.7339e-01],
         [ 2.4850e-01,  3.1471e-01, -2

In [16]:
# with torch.no_grad():
#     traced_model_3.conv1.weight.set_(((traced_model_1.conv1.weight.data + traced_model_2.conv1.weight.data)/2).get())

In [17]:
# traced_model_3.conv1.weight.data[0,0,:]

In [18]:
params = dict(traced_model_3.named_parameters())

In [19]:
params

{'conv1.weight': Parameter containing:
 tensor([[[ 2.0389e-02,  2.3583e-02,  6.6996e-02],
          [-1.6800e-01,  2.2987e-01, -2.3620e-01],
          [-2.7821e-01,  3.2265e-01, -3.7342e-02]],
 
         [[-1.1650e-01,  1.3057e-01, -2.6335e-01],
          [ 3.3119e-01,  4.4344e-02,  2.5557e-01],
          [-4.1669e-02,  1.4127e-01,  1.1641e-01]],
 
         [[ 3.1050e-01, -3.0225e-01,  3.1629e-01],
          [ 1.5597e-01, -1.5927e-01,  1.2084e-01],
          [-3.0632e-01, -3.9383e-02,  2.0956e-01]],
 
         [[ 2.0772e-01,  2.0705e-02, -2.4642e-01],
          [ 2.7372e-01, -3.2175e-01,  1.0681e-01],
          [ 2.4934e-01,  3.1727e-01,  4.3778e-02]],
 
         [[-1.8393e-01, -6.2944e-02, -2.1750e-01],
          [ 6.8667e-02,  1.1321e-01,  2.7901e-01],
          [ 1.9587e-02,  1.2802e-03, -1.5364e-01]],
 
         [[-2.7912e-01,  1.0528e-01, -2.0015e-01],
          [-2.8628e-01, -8.1481e-02,  6.0222e-02],
          [-3.2275e-01, -2.2598e-01, -2.8433e-01]],
 
         [[-2.5045e-01,  

In [20]:
traced_model_1.location

<MyWebsocketClientWorker id:A #objects local:0 #objects remote: 12>

In [21]:
traced_model_2.location

<MyWebsocketClientWorker id:A #objects local:0 #objects remote: 12>

In [22]:
model_add = my_utils.add_model(traced_model_1, traced_model_2, worker_a)

True
True
True
True
True
True


In [23]:
traced_model_1

ConvNet1D(
  original_name=ConvNet1D
  (conv1): Conv1d(original_name=Conv1d)
  (pool): MaxPool1d(original_name=MaxPool1d)
  (fc1): Linear(original_name=Linear)
  (fc2): Linear(original_name=Linear)
)

In [24]:
dict_para = dict(traced_model_1.named_parameters())
for name in dict_para:
    print(name)

conv1.weight
conv1.bias
fc1.weight
fc1.bias
fc2.weight
fc2.bias


In [25]:
for name, param in traced_model_3.named_parameters():
    print(param.data)

tensor([[[ 2.0389e-02,  2.3583e-02,  6.6996e-02],
         [-1.6800e-01,  2.2987e-01, -2.3620e-01],
         [-2.7821e-01,  3.2265e-01, -3.7342e-02]],

        [[-1.1650e-01,  1.3057e-01, -2.6335e-01],
         [ 3.3119e-01,  4.4344e-02,  2.5557e-01],
         [-4.1669e-02,  1.4127e-01,  1.1641e-01]],

        [[ 3.1050e-01, -3.0225e-01,  3.1629e-01],
         [ 1.5597e-01, -1.5927e-01,  1.2084e-01],
         [-3.0632e-01, -3.9383e-02,  2.0956e-01]],

        [[ 2.0772e-01,  2.0705e-02, -2.4642e-01],
         [ 2.7372e-01, -3.2175e-01,  1.0681e-01],
         [ 2.4934e-01,  3.1727e-01,  4.3778e-02]],

        [[-1.8393e-01, -6.2944e-02, -2.1750e-01],
         [ 6.8667e-02,  1.1321e-01,  2.7901e-01],
         [ 1.9587e-02,  1.2802e-03, -1.5364e-01]],

        [[-2.7912e-01,  1.0528e-01, -2.0015e-01],
         [-2.8628e-01, -8.1481e-02,  6.0222e-02],
         [-3.2275e-01, -2.2598e-01, -2.8433e-01]],

        [[-2.5045e-01,  3.2074e-01, -1.7339e-01],
         [ 2.4850e-01,  3.1471e-01, -2

In [26]:
# with torch.no_grad():
    # model_add.conv1.weight.set_(((traced_model_2.conv1.weight.data+traced_model_2.conv1.weight.data)/2).get())
    # model_add.conv1.weight.set_(traced_model_2.conv1.weight.data.clone().get())

In [27]:
# model_add.conv1.weight.data[0,0,:]

In [29]:
model_add.conv1.weight.data[0,0,:]

tensor([-0.0609,  0.0348, -0.3950])

In [ ]:
# traced_model_3.conv1.weight.data[0,0,:]